In [1]:
%load_ext autoreload
%autoreload 2

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
import sys
sys.path.append('../vanilla_microbotnet/')
from fd_mobilenet_v3 import FdMobileNetV3Imp2
from functools import reduce

In [4]:
model = FdMobileNetV3Imp2(classes_num=10, input_size=32, width_multiplier=0.32, mode='small')

# first layer is not trainable
first_layer = model.features[0][0]
first_layer.weight.requires_grad = False

# last fc layer is not trainable. (NOTE: there are multiple Linear units
# in the bottleneck layers. What should we do with them?)
last_linear = model.classifier[1]
weights = [last_linear.weight]
biases = [last_linear.bias]


# [p for n, p in model.named_parameters() if 'fc' in n]
# what about bias


# Only bottlenecks [1:11] are quantized
weights_to_be_quantized = [
    p for n, p in model.features[1:11].named_parameters()
    if 'conv' in n and 'lastBN' not in n and 'fc' not in n
]

# parameters of batch_norm layers
bn_weights = [model.features[0][1].weight, model.features[11][1].weight] + \
    [
        p for n, p in model.features[1:11].named_parameters()
        if 'lastBN' in n and 'weight' in n
    ]
bn_biases = [model.features[0][1].bias, model.features[11][1].bias] + \
    [
        p for n, p in model.features[1:11].named_parameters()
        if 'lastBN' in n and 'bias' in n
    ]

params = [
    {'params': weights, 'weight_decay': 1e-4},
    {'params': weights_to_be_quantized},
    {'params': biases},
    {'params': bn_weights},
    {'params': bn_biases}
]

8


In [6]:
model = FdMobileNetV3Imp2(classes_num=10, input_size=32, width_multiplier=0.32, mode='small')


8


In [39]:
min_size_quantize = 1000
only_conv = False

def is_to_be_quantized(n):
    if only_conv:
        return 'conv' in n and 'fc' not in n
    return 'conv' in n or 'fc' in n 

def is_greater_than_min_quantize(p):
    return reduce(lambda x, y: x*y, p.shape) > min_size_quantize

weights_to_be_quantized = [
    p for n, p in model.features[1:11].named_parameters()
    if is_to_be_quantized(n) and 'weight' in n
    and 'lastBN' not in n
    and is_greater_than_min_quantize(p)
    # and ( or only_conv)
]
weights_to_be_quantized

[('3.conv.0.weight', torch.Size([80, 16, 1, 1]), 1280),
 ('3.conv.3.weight', torch.Size([80, 1, 5, 5]), 2000),
 ('3.conv.5.fc.0.weight', torch.Size([20, 80]), 1600),
 ('3.conv.5.fc.2.weight', torch.Size([80, 20]), 1600),
 ('3.conv.7.weight', torch.Size([16, 80, 1, 1]), 1280),
 ('5.conv.3.weight', torch.Size([48, 1, 5, 5]), 1200),
 ('6.conv.0.weight', torch.Size([96, 16, 1, 1]), 1536),
 ('6.conv.3.weight', torch.Size([96, 1, 5, 5]), 2400),
 ('6.conv.5.fc.0.weight', torch.Size([24, 96]), 2304),
 ('6.conv.5.fc.2.weight', torch.Size([96, 24]), 2304),
 ('6.conv.7.weight', torch.Size([32, 96, 1, 1]), 3072),
 ('7.conv.0.weight', torch.Size([184, 32, 1, 1]), 5888),
 ('7.conv.3.weight', torch.Size([184, 1, 5, 5]), 4600),
 ('7.conv.5.fc.0.weight', torch.Size([46, 184]), 8464),
 ('7.conv.5.fc.2.weight', torch.Size([184, 46]), 8464),
 ('7.conv.7.weight', torch.Size([32, 184, 1, 1]), 5888),
 ('8.conv.0.weight', torch.Size([184, 32, 1, 1]), 5888),
 ('8.conv.3.weight', torch.Size([184, 1, 5, 5]), 460